In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [5]:
data = pd.read_csv('movie_bd_v5.xls')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1775,tt0760329,45000000,103071443,The Water Horse,Alexander Nathan Etel|Emily Watson|Ben Chaplin...,Jay Russell,How Do You Keep A Secret This Big?,A lonely boy discovers a mysterious egg that h...,112,Fantasy|Adventure|Family,Revolution Studios|Ecosse Films|Beacon Picture...,12/25/2007,6.2,2007
1642,tt0360201,31000000,41512007,Wimbledon,Kirsten Dunst|Paul Bettany|Sam Neill|James McA...,Richard Loncraine,She's the golden girl. He's the longshot. It's...,A pro tennis player has lost his ambition and ...,98,Comedy|Romance,Universal Pictures,9/13/2004,5.7,2004
1725,tt0486583,100000000,97838349,Fred Claus,Vince Vaughn|Paul Giamatti|John Michael Higgin...,David Dobkin,Everybody has that one relative who can't help...,Fred Claus (Vince Vaughn) and Santa Claus (Pau...,116,Action|Comedy|Drama,Silver Pictures|Warner Bros.|Jessie Nelson Pro...,11/3/2007,5.5,2007
582,tt0250224,13000000,16929123,Crazy/Beautiful,Kirsten Dunst|Jay Hernandez|Bruce Davison|Herm...,John Stockwell,When it's real. When it's right. Don't let any...,"At Pacific Palisades High, a poor Latino falls...",99,Drama|Romance,Touchstone Pictures|Ufland,6/29/2001,6.6,2001
823,tt1637706,10000000,17273593,Our Idiot Brother,Paul Rudd|Elizabeth Banks|Zooey Deschanel|Emil...,Jesse Peretz,Everybody has one.,Everybody has one. The sibling who is always j...,90,Comedy|Drama,The Weinstein Company,8/26/2011,5.7,2011


In [6]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [66]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

# Add new column profit calculated as difference between revenue and budget of the film
data = data.assign(profit = data.revenue - data.budget)

#Add new column for title length
data['title_length'] = data.apply(lambda row: len(str(row['original_title'])), axis=1)

#add new column for description words count
data['desc_len'] = data.apply(lambda row: len(str(row['overview']).split()), axis=1)

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...



#Function to split sting with many objects (e.g. actors, genres) and return them as a list of strings
def split_string_to_list(str):
    return str.split("|")

#Function to form dictionary of items (e.g genres, actors) and their frequency
# Argument dictionary is a dictionary where all items are stored
# Argument list_of_items is a list of strings containing genres info of a particular movie

def add_item_dictionary(dictionary, list_of_items):
    
    #iteratively checking all item strings in the list
    for item in list_of_items:
        if not dictionary.get(item): 
            #if item is not yet in the dictionary, adding it and setting count to 1
            dictionary[item] = 1
        else:
            #if item is in the dictionary, increasing the count
            dictionary[item] += 1 
    return

def add_single_item_dictionary(dictionary, item):
    
    #iteratively checking all item strings in the list
    if not dictionary.get(item): 
        #if item is not yet in the dictionary, adding it and setting count to 1
        dictionary[item] = 1
    else:
        #if item is in the dictionary, increasing the count
        dictionary[item] += 1 
    return


#create list of items (e.g. genres, actors) and their movie cumulative profit
def add_item_profit_dictionary(dictionary, list_of_str, profit):
    
    #iteratively checking all genres strings in the list
    for str_item in list_of_str:
        if not dictionary.get(str_item): 
            #if genre is not yet in the dictionary, adding it and setting profit to profit
            dictionary[str_item] = profit
        else:
            #if genre is in the dictionary, summing up profits
            dictionary[str_item] += profit 
    return

#define function to split date into list of day, month and year
def split_release_date(date):
    return date.split("/")

def search_max_dict(dictionary):
 
   # Find item with Max Value in Dictionary
    itemMaxValue = max(dictionary.items(), key=lambda x: x[1])
    listOfKeys = []
    # Iterate over all the items in dictionary to find keys with max value
    for key, value in dictionary.items():
        if value == itemMaxValue[1]:
            listOfKeys.append(key)
   
    return listOfKeys

def add_actor_pair_dictionary(dictionary, list_of_str):
    for x in range(len(list_of_str)-1):
        y=x+1
        while y<len(list_of_str)-1:
            tuple1 = (list_of_str[x],list_of_str[y])
            tuple2 = (list_of_str[y],list_of_str[x])
            if dictionary.get(tuple1, 0) != 0:
                dictionary[tuple1] += 1
            elif dictionary.get(tuple2, 0) != 0:
                dictionary[tuple2] += 1
            else:
                dictionary[tuple1] = 1
            y+=1
    return 


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [18]:
# в словарь вставляем номер вопроса и ваш ответ на него

answers['1'] = 'Pirates of the Caribbean: On Stranger Tides'
# если ответили верно, можете добавить комментарий со значком "+"
# +

In [14]:
data[data.budget == data.budget.max()].original_title

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

In [15]:
#ВАРИАНТ 2
data.sort_values(['budget'],ascending=False).original_title.head(1)

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

# 2. Какой из фильмов самый длительный (в минутах)?

In [67]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides'# +
answers['2'] = 'Gods and Generals'# +
answers['3'] = 'Winnie the Pooh'# +
answers['4'] = 109.6585494970884# +
answers['5'] = 107.0 # +
answers['6'] = 'Avatar'# +
answers['7'] = 'The Lone Ranger'# +
answers['8'] = 1478# +
answers['9'] = 'The Dark Knight'# +
answers['10'] = 'Mars Needs Moms'# +
answers['11'] = 'Drama'# +
answers['12'] = 'Drama'# +
answers['13'] = 'Peter Jackson'# +
answers['14'] = 'Robert Rodriguez'# +
answers['15'] = 'Chris Hemsworth'# +
answers['16'] = 'Matt Damon'# +
answers['17'] = 'Action'# +
answers['18'] = 'K-19: The Widowmaker'# +
answers['19'] = 2015# +
answers['20'] = 2014# +
answers['21'] = 'September'# +
answers['22'] = 450# +
answers['23'] = 'Peter Jackson'# +
answers['24'] = 'Four By Two Productions'# +
answers['25'] = 'Midnight Picture Show'# +
answers['26'] = ('Inside Out',
 'The Dark Knight',
 '12 Years a Slave')# +
answers['27'] = ('Daniel Radcliffe','Rupert Grint')# +

In [17]:
data[data.runtime == data.runtime.max()].original_title

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [18]:
data[data.runtime == data.runtime.min()].original_title

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [19]:
data.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [24]:
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [118]:
data[data.profit == data.profit.max()].original_title


239    Avatar
Name: original_title, dtype: object

# 7. Какой фильм самый убыточный? 

In [22]:
data[data.profit == data.profit.min()].original_title

1245    The Lone Ranger
Name: original_title, dtype: object

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [23]:
len(data[data.revenue > data.budget])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [24]:
#data[data.release_year==2008].sort_values(['profit'], ascending = False).original_title.head(1)
#
grouped_data =data[data.release_year==2008]
grouped_data[grouped_data.profit == grouped_data.profit.max()].original_title


599    The Dark Knight
Name: original_title, dtype: object

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [26]:
grouped_data= data.query('release_year >= 2012 & release_year <= 2014')
grouped_data[grouped_data.profit == grouped_data.profit.min()].original_title

1245    The Lone Ranger
Name: original_title, dtype: object

# 11. Какого жанра фильмов больше всего?

In [15]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

#Declare dictionary to store all genres and their frequency
genres_storage = {}
   
#go through all rows of dataset 
for i in data.index:
    #apply functions to genres column: split into list of genres and 
    #update the dictionary with list of genres
    add_item_dictionary(genres_storage,split_string_to_list(data.genres[i]))
   
 #Apply function to our dictionary from previous step
search_max_dict(genres_storage)

    

['Drama']

ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [36]:
#Declare dictionary to store all genres and their cumulative profits
genres_storage = {}
data_profit = data.query('profit > 0')

#go through all rows of dataset 
for i in data_profit.index:
    #apply functions to genres column: split into list of genres and 
    #update the dictionary with list of genres and  movies count
    add_item_dictionary(genres_storage,split_string_to_list(data_profit.genres[i]))
   
search_max_dict(genres_storage)


['Drama']

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [38]:
data.groupby(['director'])['revenue'].sum().sort_values(ascending=False).head(1)

director
Peter Jackson    6490593685
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [28]:
#data[data.genres.str.contains('Action',na=False)].groupby(['director']).original_title.nunique().sort_values(ascending = False).head(2)

#Declare dictionary to store directors list and number of
director_storage = {}

#create a sub dataset which satisfies conditions
data_action = data[data.genres.str.contains('Action',na=False)]

#go through all rows of sub dataset 
for i in data_action.index:
    #apply functions to directors column: split into list of directos and 
    #update the dictionary with list of directors and movies count
    add_item_dictionary(director_storage,split_string_to_list(data_action.director[i]))
   
#Apply function to our dictionary from previous step
search_max_dict(director_storage)



['Robert Rodriguez']

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [31]:
#Declare dictionary to store actors list and  movies cumulative revenue
actors_profit_storage = {}

#create a sub dataset which satisfies conditions
data2012 = data.query('release_year == 2012')

#go through all rows of sub dataset 
for i in data2012.index:
    #apply functions to actors column: split into list of actors and 
    #update the dictionary with list of genres and  profit
    add_item_profit_dictionary(actors_profit_storage,split_string_to_list(data2012.cast[i]),data2012.revenue[i])
   
#Apply function to our dictionary from previous step
search_max_dict(actors_profit_storage)




['Chris Hemsworth']

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [34]:
#filter dataset for high budget movies only
data_profit_movies = data[data.budget > data.budget.mean()]

#Declare dictionary to store all actors and count of movies
actors_storage = {}
   
#go through all rows of dataset 
for i in data_profit_movies.index:
    #apply functions to actors column: split into list of actors and 
    #update the dictionary with list of actors and  movie count
    add_item_dictionary(actors_storage,split_string_to_list(data_profit_movies.cast[i]))
   
#Apply function to our dictionary from previous step
search_max_dict(actors_storage)



['Matt Damon']

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [6]:
data_cage_movies = data[data.cast.str.contains('Nicolas Cage',na=False)]
#Declare dictionary to store all genres and their frequency
genres_storage = {}
   
#go through all rows of dataset 
for i in data_cage_movies.index:
    #apply functions to genres column: split into list of genres and 
    #update the dictionary with list of genres
    add_item_dictionary(genres_storage,split_string_to_list(data_cage_movies.genres[i]))
   
#Apply function to our dictionary from previous step
search_max_dict(genres_storage)

['Action']

# 18. Самый убыточный фильм от Paramount Pictures

In [43]:

#filter dataset for unprofitable movies only
data_nonprofit = data[data.production_companies.str.contains('Paramount Pictures',na = False)].query('profit <= 0')
data_nonprofit[data_nonprofit.profit == data_nonprofit.profit.min()].original_title



925    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [44]:
grouped_df=data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False)
grouped_df.iloc[[0]]


release_year
2015    25449202382
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [45]:
data_profit = data[data.production_companies.str.contains('Warner Bros',na = False)].groupby(['release_year'])['profit'].sum().sort_values(ascending=False)
data_profit.iloc[[0]]

release_year
2014    2295464519
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [16]:
#Declare dictionary to store all months and amount of movies
month_movie_storage = {}



#go through all rows of dataset 
for i in data.index:
    #apply functions to release_date column: split into list of genres and 
    #update the dictionary with list of genres
    list=[]
    list = split_release_date(data.release_date[i])
    add_single_item_dictionary(month_movie_storage,int(list[0]))
   
 #Apply function to our dictionary from previous step
search_max_dict(month_movie_storage)




[9]

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [17]:
month_movie_storage[6]+month_movie_storage[7]+month_movie_storage[8]

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [18]:
#Declare dictionary to store all directos and amount of movies released in winter
winter_movie_director_storage = {}



#go through all rows of dataset 
for i in data.index:
    #apply functions to release_date column: split into list of genres and 
    #update the dictionary with list of genres
    list=[]
    list = split_release_date(data.release_date[i])
    #check that only winter months 1,2 and 12 are analysed
    if int(list[0]) == 12 or int(list[0]) <=2:
        add_item_dictionary(winter_movie_director_storage,split_string_to_list(data.director[i]))
        
search_max_dict(winter_movie_director_storage)

['Peter Jackson']

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [51]:
#Assumption - longest titles on average
#Declare dictionary to store all production companies and their frequency
prod_storage = {}
prod_length = {}
   
#go through all rows of dataset 
for i in data.index:
    #apply functions to production_companies column: split into list of production_companies and 
    #update the dictionary with list of production_companies
    add_item_dictionary(prod_storage,split_string_to_list(data.production_companies[i]))
    add_item_profit_dictionary(prod_length,split_string_to_list(data.production_companies[i]),data.title_length[i])
   

max = 0
# Iterate over all the items in dictionary to find keys with max value
for key, value in prod_storage.items():
    #dict.get('Education', "Never")
    if prod_length.get(key,1)/value > max:
        max = prod_length.get(key) / value
        max_prod = key
max_prod   


'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [62]:
#Declare dictionary to store all production companies and their frequency
prod_storage = {}
prod_desc_length = {}
   
#go through all rows of dataset 
for i in data.index:
    #apply functions to production_companies column: split into list of production_companies and 
    #update the dictionary with list of production_companies and movies count
    add_item_dictionary(prod_storage,split_string_to_list(data.production_companies[i]))
     #apply functions to production_companies column: split into list of production_companies and 
    #update the dictionary with list of production_companies and total amount of words of all their movies descriptions
    add_item_profit_dictionary(prod_desc_length,split_string_to_list(data.production_companies[i]),data.desc_len[i])
   

max = 0
# Iterate over all the items in both dictionary to find keys with max value
for key, value in prod_storage.items():
    #check if average descrption length (cumulative length divided by number of movies) is 
    #greater then known max
     if prod_desc_length.get(key,0) / value > max:
        #update max
        max = prod_desc_length.get(key) / value
        #store production company name
        max_prod = key
#display company name
max_prod  



'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [63]:

data.sort_values(['vote_average'],ascending=False).original_title.head(int(len(data)*0.01)).tolist()


['The Dark Knight',
 'Interstellar',
 'The Imitation Game',
 'Inside Out',
 'Room',
 'The Wolf of Wall Street',
 'Gone Girl',
 '12 Years a Slave',
 'Guardians of the Galaxy',
 'The Lord of the Rings: The Return of the King',
 'Memento',
 'Inception',
 'The Pianist',
 'The Grand Budapest Hotel',
 'Her',
 'Spotlight',
 'Big Hero 6',
 'The Fault in Our Stars']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [116]:
#Declare dictionary to store all actors pair companies and amount of their joint movies
actor_pair_movies={}
      
#go through all rows of dataset 
for i in data.index:
    #apply functions to cast column: split into list of actors and 
    #update the dictionary with list of actors
    add_actor_pair_dictionary(actor_pair_movies,split_string_to_list(data.cast[i]))
    


sorted_x = dict(sorted(actor_pair_movies.items(), key=lambda item: item[1], reverse= True))
sorted_x

                    
        

{('Daniel Radcliffe', 'Rupert Grint'): 8,
 ('Daniel Radcliffe', 'Emma Watson'): 8,
 ('Rupert Grint', 'Emma Watson'): 8,
 ('Vin Diesel', 'Paul Walker'): 5,
 ('Kristen Stewart', 'Robert Pattinson'): 5,
 ('Kristen Stewart', 'Taylor Lautner'): 5,
 ('Robert Pattinson', 'Taylor Lautner'): 5,
 ('Johnny Depp', 'Helena Bonham Carter'): 5,
 ('Jennifer Lawrence', 'Josh Hutcherson'): 4,
 ('Jennifer Lawrence', 'Liam Hemsworth'): 4,
 ('Jennifer Lawrence', 'Woody Harrelson'): 4,
 ('Josh Hutcherson', 'Liam Hemsworth'): 4,
 ('Josh Hutcherson', 'Woody Harrelson'): 4,
 ('Liam Hemsworth', 'Woody Harrelson'): 4,
 ('Adam Sandler', 'Kevin James'): 4,
 ('Hugh Jackman', 'Patrick Stewart'): 4,
 ('Matt Damon', 'George Clooney'): 4,
 ('Ben Stiller', 'Owen Wilson'): 4,
 ('Tobin Bell', 'Costas Mandylor'): 4,
 ('Tobin Bell', 'Betsy Russell'): 4,
 ('Costas Mandylor', 'Betsy Russell'): 4,
 ('Vin Diesel', 'Jordana Brewster'): 4,
 ('Paul Walker', 'Jordana Brewster'): 4,
 ('Mike Myers', 'Eddie Murphy'): 4,
 ('Mike Myers'

# Submission

In [65]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': 109.6585494970884,
 '5': 107.0,
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': 1478,
 '9': 'The Dark Knight',
 '10': 'Mars Needs Moms',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20': 2014,
 '21': 'September',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ('Inside Out', 'The Dark Knight', '12 Years a Slave'),
 '27': ('Daniel Radcliffe', 'Rupert Grint')}

In [20]:
# и убедиться что ни чего не пропустил)
len(answers)

27